In [2]:
from keras.models import Sequential
from keras.layers import LSTM, TimeDistributed, RepeatVector, Dense
import numpy as np
import wandb
from wandb.keras import WandbCallback
import io
import unicodedata
import re

In [ ]:
wandb.init()
config = wandb.config

In [3]:
# Data input
# Use my own database
path_to_file='assets/human-robot.txt'

In [4]:
# Converts the unicode file to ascii
def unicode_to_ascii(s):
  return ''.join(c for c in unicodedata.normalize('NFD', s)
      if unicodedata.category(c) != 'Mn')


def preprocess_sentance(w):
  w = unicode_to_ascii(w.lower().strip())

  # creating a space between a word and the punctuation following it
  # eg: "he is a boy." => "he is a boy ."
  # Reference:- https://stackoverflow.com/questions/3645931/python-padding-punctuation-with-white-spaces-keeping-punctuation
  w = re.sub(r"([?.!,¿])", r" \1 ", w)
  w = re.sub(r'[" "]+', " ", w)

  # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")
  w = re.sub(r"[^a-zA-Z?.!,¿]+", " ", w)

  w = w.strip()

  # adding a start and an end token to the sentence
  # so that the model know when to start and stop predicting.
  w = '<start> ' + w + ' <end>'
  return w

In [5]:
class CharacterTable (object):
    """Given a set of characters:
    + Encode them to a one hot integer representation
    + Decode the on hot integer representation to thier character output
    + Decode a vector of probabilities to their character output
    """
    def __init__(self, chars):
        """Initialize character table
        # Arguments
           chars: Characters that can appear in the input
        """
        self.chars = sorted(set(chars))
        self.char_indices = dict((c, i) for i, c in enumerate(self.chars))
        self.indicies_char = dict((i, c) for i, c in enumerate(slef.chars))

        

    def encode(self, C, num_rows):
        """One hot encode given string C.
        # Arguments
            num_rows: Number of rows in the returned one hot encoding. This is 
                used to keep the ' of the rows for each data the same.
        """
        x = np.zeros((num_rows, len(self.chars)))
        for i, c in enumerate(c):
            if calc_argmax:
                x = x.argmax(axis=-1)
            return ''.join(self.indicies_char[x] for x in x)


        def decode(self, x, calc_argmax=True):
        if calc_argmax:
            x = x.argmax(axis=-1)
        return ''.join(self.indices_char[x] for x in x)

IndentationError: expected an indented block (<ipython-input-5-16f4380aa300>, line 32)

In [6]:
path = 'assets/human-robot.txt'
lines = io.open(path, encoding='UTF-8').read().split('\n')
word_pairs = [[preprocess_sentance(w) for w in l.split('\t')] for l in lines[:1000]]

In [30]:
class Data_layout (object):
    def __init__(self, path):
        self.inp, self.targ = self.generate_word_pairs(path)

        #Generate vocab list
        self.inp_vocab = self.generate_vocab(self.inp)
        self.targ_vocab = self.generate_vocab(self.targ)

        self.input_features_dict = dict([(token, i) for i, token in enumerate(self.inp_vocab)])
        self.reverse_input_features_dict = dict((i, token) for token, i in self.input_features_dict.items())

        self.inp_max_length = len(max(self.inp, key=len))
        self.targ_max_length = len(max(self.targ, key=len))

        #Generate one hot encoded arrays
        #for i, sentence in enumerate(self.inp):
            #self.x = self.encode(sentence, self.inp_max_length, self.reverse_input_features_dict)


    def generate_word_pairs(self, path):
        # Split the dataset into two parts
        lines = io.open(path, encoding='UTF-8').read().split('\n')
        word_pairs = [[preprocess_sentance(w) for w in l.split('\t')] for l in lines[:1000]]
        # seperate the word pairs out
        return zip(*word_pairs)

    def generate_vocab(self,array):
        tokens = set()
        for line in array:
            for token in re.findall(r"[\w']+|[^\s\w]", line):
                if token not in tokens:
                    tokens.add(token)
        return tokens

    

    def encode (self, C, num_rows, vocab):
        x = np.zeros((num_rows, len(vocab)))
        for i, c in enumerate(C):
            x[i, self.input_features_dict[c]] = 1
        return x

In [31]:
path_to_file='assets/human-robot.txt'
dataset = Data_layout(path_to_file)
print(dataset.inp_max_length)
x = np.zeros((len(dataset.inp), dataset.inp_max_length, len(dataset.inp_vocab)), dtype=np.bool)
for i, sentence in enumerate (dataset.inp):
    x[i] = dataset.encode(sentence, dataset.inp_max_length, dataset.reverse_input_features_dict)
x


1104


KeyError: ' '

In [ ]:
# config.traning_size = 1000
# config.digits = 5
config.hidden_size = 128
config.batch_size = 128

x, y,  = load_dataset(path_to_file, 1000)

# maxlen = config.digits + 1 + config.digits

ctable = CharacterTable(chars)


# Things needed to update the model to work with my data
* chars: is the vocabulary of the input data
* digits: the number of unique "tokens" are in the input text
* maxlen: the longest possible length of a input vector
* x: list of all input data one hot integer array
* y: list of all output data one hot integer array

In [ ]:
model = Sequential()
# Needs to be modified
mode.add(LSTM(config.bidden_size, input_shape=(maxlen, len(chars))))
model.add(RepeatVector(config.digits+1))
model.add(LSTM(config.hidden_size, return_sequences=True))
model.add(TimeDistributed(Dense(len(chars), activation='softmax')))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

In [ ]:
for iteration in range(1, 200):
    print()
    print('-' * 50)
    print('Iterations', iteration)
    model.fit(x_train, y_train, batch_size=config.batch_size=, epochs=1, validation_data=(x_val, y_val), callbacks=[WandbCallback()])
    
    # Select 10 sampless from the validation set at random so we can visualize errors
    for i in range(10):
        ind = np.random.randint(0, len(x_val))
        rowx, rowy - x_val[np.array([ind])], y_val[np.array([ind])]
        preds = model.predict_classes(rowx verbose=0)
        q = ctable.decode(rowy[0])
        guess = ctable.decode(preds[0], calc_argmax=False)
        print('Q', q, end=' ')
        print('T', correct, end=' ')
        if correct == guess:
            print('☑', end=' ')
        else:
            print('☒', end=' ')
        print(guess)